In [5882]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [5883]:
company_name = 'mastek'.upper()

In [5884]:
particulars = 'particulars'

In [5885]:
share_price_in = 1000000

In [5886]:
def calculate_growth(current , previous) :
    if current == 0 and previous == 0 or current == 0 and previous != 0:
        return 0
    if current != 0 and previous == 0 :
        return current / 100
    return ((current/previous) - 1)
       

In [5887]:
def calculate_growth_series(curr_df,prev_df):
    temp_df = curr_df.copy()
    for index in curr_df.index :
        temp_df.loc[index] = calculate_growth(curr_df.loc[index],prev_df[index])
    return temp_df
    

In [5888]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [5889]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [5890]:
def round_off_to(num , limit) :
    return round(num,limit)

In [5891]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [5892]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [5893]:
def calculate_ratio(dividend , divisor) : 
    return dividend.div(divisor.values).apply(lambda x: round(x,2))

In [5894]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [5895]:
def days_turnover(ratio) :
    if ratio == '-' :
        return '-'
    return round((365 / ratio),2)

In [5896]:
def get_df_items(df , key_name , start_col) :
     filter = df[particulars].apply(lambda x: x.strip().lower()).eq(key_name.lower())
     temp_df = pd.DataFrame(df.loc[filter,start_col:])
     temp_df.reset_index(drop=True,inplace=True) 
     return temp_df

In [5897]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [5898]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [5899]:
income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [5900]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [5901]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [5902]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [5903]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [5904]:
temp_df = temp_df.apply(get_share_price_in)

In [5905]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [5906]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [5907]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue

In [5908]:
total_rev = income_statement[particulars].index == 0
income_statement.loc[total_rev,particulars] = income_statement.loc[total_rev,particulars].values[0].strip() 

In [5909]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [5910]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [5911]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [5912]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [5913]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [5914]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [5915]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [5916]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [5917]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is_style = vertical_analysis_is
vertical_analysis_is_style = vertical_analysis_is_style.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 
fil = None
templist = None

In [5918]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None
balance_sheet_va_results_till = None
filt = None

In [5919]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column])

In [5920]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va_style = balance_sheet_va
balance_sheet_va_style = balance_sheet_va_style.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil = None
bs_templist = None

In [5921]:
#vertical_analysis_is_style

In [5922]:
#balance_sheet_va_style

In [5923]:
income_statement_ha = pd.DataFrame(columns= income_statement.columns)

In [5924]:
income_statement_ha[particulars] = income_statement[particulars]

In [5925]:
income_statement_ha.fillna(0 , inplace=True)

In [5926]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0,4])
for index , column in enumerate(income_statement_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = income_statement.columns[index-1]
        income_statement_ha.loc[temp_ha_filt,column] = calculate_growth_series(income_statement.loc[temp_ha_filt,column],income_statement.loc[temp_ha_filt,prevColumn])
        

In [5927]:
income_statement_ha = income_statement_ha[temp_ha_filt]

In [5928]:
#income_statement_ha.replace([float('inf'), float('-inf')], None, inplace=True)
#income_statement_ha.fillna(0 , inplace=True)

In [5929]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0])
iha_templist = income_statement_ha.loc[temp_ha_filt,particulars].values
income_statement_ha_style = income_statement_ha
income_statement_ha_style = income_statement_ha_style.style.apply(row_style,list=iha_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [5930]:
balance_sheet_ha_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_ha_results_till = balance_sheet[balance_sheet_ha_results_till].index.values[0]
bs_ha_temp = balance_sheet.loc[:balance_sheet_ha_results_till]
bs_ha_temp = pd.DataFrame(bs_ha_temp)
filt = bs_ha_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_ha = pd.DataFrame(bs_ha_temp[filt])

In [5931]:
for index , column in enumerate(balance_sheet_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = balance_sheet_ha.columns[index-1]
        balance_sheet_ha.loc[:,column] = calculate_growth_series(balance_sheet.loc[:,column],balance_sheet.loc[:,prevColumn])
balance_sheet_ha[start_col_balance_sheet] = 0.0
prevColumn = None

In [5932]:
bs_fil_ha = balance_sheet_ha[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ha = balance_sheet_ha.loc[bs_fil_ha,particulars].values
balance_sheet_ha_style = balance_sheet_ha
balance_sheet_ha_style = balance_sheet_ha_style.style.apply(row_style,list=bs_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ha = None
bs_templist_ha = None

In [5933]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_ha = pd.DataFrame(cashflow_statement[cashflow_statement_filt])
cash_flow_stmt_ha.reset_index(inplace=True, drop=True)
cash_flow_stmt_ha_temp = cash_flow_stmt_ha.copy()

In [5934]:
for index , column in enumerate(cash_flow_stmt_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = cash_flow_stmt_ha.columns[index-1]
        cash_flow_stmt_ha.loc[:,column] = calculate_growth_series(cash_flow_stmt_ha_temp.loc[:,column],cash_flow_stmt_ha_temp.loc[:,prevColumn])
cash_flow_stmt_ha[start_col_cashflow] = 0.0
cash_flow_stmt_ha_temp = None
cashflow_statement_filt = None
prevColumn = None

In [5935]:
cf_fil_ha = cash_flow_stmt_ha[particulars].apply(count_leading_space).isin([0])
cf_templist_ha = cash_flow_stmt_ha.loc[cf_fil_ha,particulars].values
cash_flow_stmt_ha_style = cash_flow_stmt_ha
cash_flow_stmt_ha_style = cash_flow_stmt_ha_style.style.apply(row_style,list=cf_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ha = None
cf_templist_ha = None

In [5936]:
#income_statement_ha_style

In [5937]:
#balance_sheet_ha_style

In [5938]:
#cash_flow_stmt_ha_style

In [5939]:
is_trend_base = income_statement[start_col]

In [5940]:
income_statement_trend = pd.DataFrame(columns=income_statement.columns)
income_statement_trend[particulars] = income_statement[particulars]
income_statement_trend[start_col] = 0
income_statement_trend.fillna(0,inplace=True)

In [5941]:
for index , column in enumerate(income_statement_trend.columns) :
    if index < 2 :
        pass;
    else:
        income_statement_trend.loc[:,column] = calculate_growth_series(income_statement.loc[:,column],is_trend_base)
is_trend_base = None      

In [5942]:
temp_ta_filt = income_statement_trend[particulars].apply(count_leading_space).isin([0])
ita_templist = income_statement_trend.loc[temp_ta_filt,particulars].values
income_statement_trend_style = income_statement_trend
income_statement_trend_style = income_statement_trend_style.style.apply(row_style,list=ita_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [5943]:
temp_balance = balance_sheet.loc[:balance_sheet_ha_results_till]

In [5944]:
bs_trend_base = temp_balance[start_col_balance_sheet]
balance_sheet_trend = pd.DataFrame(columns=temp_balance.columns)
balance_sheet_trend[particulars] = temp_balance[particulars]
balance_sheet_trend[start_col] = 0
balance_sheet_trend.fillna(0,inplace=True)

In [5945]:
for index , column in enumerate(balance_sheet_trend.columns) :
    if index < 2 :
        pass;
    else:
        balance_sheet_trend.loc[:,column] = calculate_growth_series(temp_balance.loc[:,column],bs_trend_base)
temp_balance = None
bs_trend_base = None

In [5946]:
bs_fil_ta = balance_sheet_trend[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ta = balance_sheet_trend.loc[bs_fil_ta,particulars].values
balance_sheet_trend_style = balance_sheet_trend
balance_sheet_trend_style = balance_sheet_trend_style.style.apply(row_style,list=bs_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ta = None
bs_templist_ta = None

In [5947]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_trend = pd.DataFrame(columns=cashflow_statement.columns)
cash_flow_stmt_trend_temp = cashflow_statement[cashflow_statement_filt].copy()
cash_flow_stmt_trend_temp.reset_index(inplace=True,drop=True)
cash_flow_stmt_trend[particulars] = cash_flow_stmt_trend_temp[particulars]
cash_flow_stmt_trend[start_col_cashflow] = 0

In [5948]:
cf_trend_base = cash_flow_stmt_trend_temp[start_col_cashflow]

In [5949]:
for index , column in enumerate(cash_flow_stmt_trend.columns) :
    if index < 2 :
        pass;
    else:
        cash_flow_stmt_trend.loc[:,column] = calculate_growth_series(cash_flow_stmt_trend_temp.loc[:,column],cf_trend_base)
cash_flow_stmt_trend_temp = None
cf_trend_base = None

In [5950]:
cf_fil_ta = cash_flow_stmt_trend[particulars].apply(count_leading_space).isin([0])
cf_templist_ta = cash_flow_stmt_trend.loc[cf_fil_ta,particulars].values
cash_flow_stmt_trend_style = cash_flow_stmt_trend
cash_flow_stmt_trend_style = cash_flow_stmt_trend.style.apply(row_style,list=cf_templist_ta,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow trend analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ta = None
cf_templist_ta = None

In [5951]:
#income_statement_trend_style

In [5952]:
#balance_sheet_trend_style

In [5953]:
#cash_flow_stmt_trend_style

In [5954]:
current_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current assets')
current_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('total current liabilities')

In [5955]:
current_assets = balance_sheet.loc[current_assets_filter,start_col_balance_sheet:].copy()
current_liabilities = balance_sheet.loc[current_liabilities_filter,start_col_balance_sheet:].copy()

In [5956]:
solvency_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [5957]:
solvency_ratio[particulars] = ['current ratio' , 'quick ratio', 'cash ratio']

In [5958]:
solvency_ratio.loc[0:0,start_col_balance_sheet:] = calculate_ratio(current_assets,current_liabilities).values

In [5959]:
inventories_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('inventories')

In [5960]:
inventories = balance_sheet.loc[inventories_filter,start_col_balance_sheet:].copy()
if len(inventories.value_counts().values) == 0 :
    data = [[0] * len(inventories.columns)]
    inventories = pd.DataFrame(data=data , columns=inventories.columns).copy()
quick_assets = current_assets - inventories.values

In [5961]:
solvency_ratio.loc[1:1,start_col_balance_sheet:] =  calculate_ratio(quick_assets,current_liabilities).values

In [5962]:
cash_and_ceq_filter = balance_sheet[particulars].apply(lambda x: x.strip().lower()).eq('cash and cash equivalents')

In [5963]:
cash_and_ceq = balance_sheet.loc[cash_and_ceq_filter,start_col_balance_sheet:].copy()

In [5964]:
solvency_ratio.loc[2:2,start_col_balance_sheet:] = calculate_ratio(cash_and_ceq,current_liabilities).values

In [5965]:
solvency_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,current ratio,2.42,2.5,2.7,3.02,2.26,2.87,1.85,1.87,1.78
1,quick ratio,2.42,2.5,2.7,3.02,2.26,2.87,1.85,1.87,1.78
2,cash ratio,0.53,0.86,0.86,0.46,0.46,0.55,0.47,0.81,0.85


In [5966]:
turnover_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [5967]:
turnover_ratio[particulars] = ['Receivable turnover' , 'Inventory turnover', 'Payables turnover']

In [5968]:
#turnover_ratio.drop([start_col_balance_sheet] , axis=1 , inplace=True)

In [5969]:
account_receivables = get_df_items(balance_sheet,'Trade and Other Receivables, Current' , start_col_balance_sheet)

In [5970]:
invetory = get_df_items(balance_sheet,'Inventories' , start_col_balance_sheet)

In [5971]:
cogs = get_df_items(income_statement,'Cost of Revenue' , start_col).apply(abs)

In [5972]:
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        ar_mean = account_receivables.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[0:0,column] = calculate_ratio(total_revenue.loc[:,column],ar_mean).values


In [5973]:
for i , column in enumerate(balance_sheet.columns):
    if(len(invetory.value_counts()) == 0) :
        turnover_ratio.loc[1:1,start_col_balance_sheet:] = '-'
        break
    elif i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        inv_mean = invetory.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[1:1,column] = calculate_ratio(cogs.loc[:,column],inv_mean).values    

In [5974]:
bl_is_col_diff = len(balance_sheet.columns) - len(income_statement.columns)

In [5975]:
cogs_ap = get_df_items(income_statement,'Cost of Revenue',start_col).iloc[:,:bl_is_col_diff].apply(abs)
invetories_ap = get_df_items(balance_sheet,'inventories',start_col_balance_sheet)
if len(invetories_ap.value_counts().values) == 0 :
    data = [[0] * len(invetories_ap.columns)]
    invetories_ap = pd.DataFrame(data=data , columns=invetories_ap.columns)
purchases = pd.DataFrame(data=[ [0] * len(invetories_ap.columns) ],columns=invetories_ap.columns.values)
for i , column in enumerate(invetories_ap.columns) :
    if i > 0 :
        currentCol = column
        prevColumn = invetories_ap.columns[i-1]
        inventories_diff = invetories_ap[currentCol] - invetories_ap[prevColumn]
        purchases[currentCol] = cogs_ap[currentCol] + inventories_diff.values    

In [5976]:
account_payables = get_df_items(balance_sheet,'Payables and Accrued Expenses, Current',start_col_balance_sheet)
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i - 1]
        ap_mean = account_payables.loc[:,prevColumn:column].mean(axis=1)
        turnover_ratio.loc[2:2,column] = calculate_ratio(purchases.loc[:,column],ap_mean).values
turnover_ratio.dropna(how='all' , axis=1,inplace=True)

In [5977]:
#turnover_ratio.drop([start_col_balance_sheet] , inplace=True , axis=1)

In [5978]:
days_turnover_ratio = pd.DataFrame(columns=turnover_ratio.columns)

In [5979]:
days_turnover_ratio[particulars] = ['Receivable turnover days' , 'Inventory turnover days' , 'Payables turnover days']

In [5980]:
days_turnover_ratio.iloc[:,1:] = turnover_ratio.iloc[:,1:].applymap(days_turnover)

In [5981]:
turnover_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Receivable turnover,NaN,4.41,2.71,3.6,4.36,4.3,2.9,3.39,3.53
1,Inventory turnover,-,-,-,-,-,-,-,-,-
2,Payables turnover,NaN,6.99,4.03,5.56,6.51,6.13,2.97,3.12,3.29


In [5982]:
days_turnover_ratio

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Receivable turnover days,NaN,82.77,134.69,101.39,83.72,84.88,125.86,107.67,103.4
1,Inventory turnover days,-,-,-,-,-,-,-,-,-
2,Payables turnover days,NaN,52.22,90.57,65.65,56.07,59.54,122.9,116.99,110.94


In [5983]:
sc = days_turnover_ratio.columns[1]
receivable_days = get_df_items(days_turnover_ratio,'Receivable turnover days',sc)
inventory_days = get_df_items(days_turnover_ratio,'Inventory turnover days',sc).replace({'-' : 0})
payable_days = get_df_items(days_turnover_ratio,'Payables turnover days',sc)
ccy = inventory_days + receivable_days - payable_days
cash_conversion_cyle = pd.DataFrame(ccy) 


In [5984]:
ccy

,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NaN,30.55,44.12,35.74,27.65,25.34,2.96,-9.32,-7.54


In [5985]:
operating_efficiency_ratio = pd.DataFrame(columns=balance_sheet.columns)

In [5986]:
bl_is_col_diff = len(balance_sheet.columns) - len(income_statement.columns)

In [5987]:
operating_efficiency_ratio[particulars] = ['Total asset turnover' , 'Net fixed asset turnover' , 'Equity turnover']

In [5988]:
operating_efficiency_total_revenue = get_df_items(income_statement,'Total Revenue' , start_col).iloc[:,:bl_is_col_diff]
operating_efficiency_total_assets = get_df_items(balance_sheet,'Total Assets' , start_col_balance_sheet)
operating_efficiency_ppe = get_df_items(balance_sheet,'Net Property, Plant and Equipment' , start_col_balance_sheet)
operating_efficiency_equity = get_df_items(balance_sheet,'Equity Attributable to Parent Stockholders' , start_col_balance_sheet)
for i , column in enumerate(balance_sheet.columns):
    if i > 1 :
        prevColumn = balance_sheet.columns[i-1]
        ta_mean = operating_efficiency_total_assets.loc[:,prevColumn:column].mean(axis=1)
        ppe_mean = operating_efficiency_ppe.loc[:,prevColumn:column].mean(axis=1)
        equity_mean = operating_efficiency_equity.loc[:,prevColumn:column].mean(axis=1)
        operating_efficiency_ratio.loc[0:0,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],ta_mean).values
        operating_efficiency_ratio.loc[1:1,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],ppe_mean).values
        operating_efficiency_ratio.loc[2:2,column] = calculate_ratio(operating_efficiency_total_revenue.loc[:,column],equity_mean).values
operating_efficiency_ratio.dropna(how='all',axis=1,inplace=True)

In [5989]:
operating_efficiency_ratio

,particulars,2015,2016,2017,2018,2019,2020,2021,2022
0,Total asset turnover,1.28,0.84,1.17,1.19,1.13,0.74,0.82,0.91
1,Net fixed asset turnover,14.46,9.51,12.87,17.87,21.77,17.86,25.78,32.89
2,Equity turnover,1.75,1.15,1.68,1.82,1.63,1.42,2.09,2.26


In [5990]:
operating_profitability_ratio_income = pd.DataFrame(columns=income_statement.columns)

In [5991]:
operating_profitability_ratio_income[particulars] = ['Gross profit margin' , 'Operating profit margin', 'Net profit margin' , 'Return on total assets','Return on total equity','Return on owners equity' , 'Leverage factor']

In [5992]:
operating_profitability_gross_profit = get_df_items(income_statement,'Gross Profit' , start_col).apply(abs)
operating_profitability_revenue = get_df_items(income_statement,'Total Revenue' , start_col).apply(abs)
operating_income_expense = get_df_items(income_statement,'Operating Income/Expenses' , start_col).apply(abs)
other_operating_income_expense = get_df_items(income_statement,'Other Income/Expense, Operating' , start_col).apply(abs)
EBIT = operating_profitability_gross_profit - (operating_income_expense - other_operating_income_expense)
net_income = get_df_items(income_statement,'Net Income before Extraordinary Items and Discontinued Operations' , start_col)
net_income_owners = get_df_items(income_statement,'Net Income after Non-Controlling/Minority Interests' , start_col)
operating_profitability_total_assets = get_df_items(balance_sheet,'Total Assets' , start_col_balance_sheet)
total_equity = get_df_items(balance_sheet,'Total Equity' , start_col_balance_sheet)
total_owners_equity = get_df_items(balance_sheet,'Equity Attributable to Parent Stockholders' , start_col_balance_sheet)

In [5993]:
operating_profitability_ratio_income.loc[0:0,start_col:] = vertical_analysis(operating_profitability_gross_profit,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [5994]:
operating_profitability_ratio_income.loc[1:1,start_col:] = vertical_analysis(EBIT,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [5995]:
operating_profitability_ratio_income.loc[2:2,start_col:] = vertical_analysis(net_income,operating_profitability_revenue).apply(lambda x:round(x,2)).values

In [5996]:
operating_profitability_balance_columns = balance_sheet.loc[:,start_col:].columns

In [5997]:

for i , column in enumerate(operating_profitability_balance_columns) :
    if i > 0 :
        prevColumn =  operating_profitability_balance_columns[i-1]
        assets_mean = operating_profitability_total_assets.loc[:,prevColumn:column].mean(axis=1)
        equity_mean = total_equity.loc[:,prevColumn:column].mean(axis=1)
        owners_equity_mean = total_owners_equity.loc[:,prevColumn:column].mean(axis=1)
        operating_profitability_ratio_income.loc[3:3,column] = vertical_analysis(EBIT.loc[:,column],assets_mean).apply(lambda x:round(x,2)).values
        operating_profitability_ratio_income.loc[4:4,column] = vertical_analysis(net_income.loc[:,column],equity_mean).apply(lambda x:round(x,2)).values
        operating_profitability_ratio_income.loc[5:5,column] = vertical_analysis(net_income_owners.loc[:,column],owners_equity_mean).apply(lambda x:round(x,2)).values
        operating_profitability_ratio_income.loc[6:6,column] = vertical_analysis(assets_mean,equity_mean).apply(lambda x:round(x,2)).values
        
        

In [5998]:
operating_profitability_ratio_income.fillna('-' , inplace=True)

In [5999]:
operating_profitability_ratio_income

,particulars,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Gross profit margin,0.39,0.41,0.47,0.44,0.41,0.43,0.48,0.51,0.52,0.0
1,Operating profit margin,0.14,0.11,0.10,0.14,0.16,0.15,0.15,0.22,0.22,-0.05
2,Net profit margin,0.06,0.02,0.03,0.07,0.09,0.10,0.11,0.15,0.15,0.15
3,Return on total assets,-,0.14,0.09,0.16,0.19,0.17,0.11,0.18,0.20,-
4,Return on total equity,-,0.04,0.03,0.11,0.16,0.16,0.14,0.26,0.29,-
5,Return on owners equity,-,0.03,0.03,0.11,0.16,0.16,0.14,0.25,0.31,-
6,Leverage factor,-,1.37,1.38,1.43,1.53,1.44,1.76,2.13,2.11,-


In [6000]:
dupont = pd.DataFrame(columns=operating_efficiency_ratio.columns)
dupont[particulars] = ['Total asset turnover' , 'Net profit margin' , 'Leverage factor' , 'DuPont ROE']

In [6001]:
for i ,column in enumerate(dupont.columns) :
    if i > 0 :
        asset_turnover = operating_efficiency_ratio.loc[0:0 , column]
        leverage_factor = operating_profitability_ratio_income.loc[6:6,column]
        net_margin = operating_profitability_ratio_income.loc[2:2,column]
        Dupont = (asset_turnover * net_margin.values * leverage_factor.values)[0]
        dupont.loc[0:0,column] = asset_turnover.values
        dupont.loc[1:1,column] = net_margin.values
        dupont.loc[2:2,column] = leverage_factor.values
        dupont.loc[3:3,column] = round(Dupont,2)
        
        

In [6002]:
dupont

,particulars,2015,2016,2017,2018,2019,2020,2021,2022
0,Total asset turnover,1.28,0.84,1.17,1.19,1.13,0.74,0.82,0.91
1,Net profit margin,0.02,0.03,0.07,0.09,0.1,0.11,0.15,0.15
2,Leverage factor,1.37,1.38,1.43,1.53,1.44,1.76,2.13,2.11
3,DuPont ROE,0.04,0.03,0.12,0.16,0.16,0.14,0.26,0.29
